## Import Necessary Libraries 

In [1]:
import pandas as pd
import matplotlib as plt
from  sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf 

### Load data

In [12]:
dataDir = "./urban_mobility_data_past_year.csv"
df = pd.read_csv(dataDir)
df

,timestamp,congestion_level,weather_conditions,road_type,vehicle_type,temperature,humidity,wind_speed,precipitation,population_density
0,2022-08-01 00:00:00,0.843396,Rainy,City Street,Car,26.375612,46.517794,8.230188,7.786502,3459.059905
1,2022-08-01 00:29:00,0.858039,Rainy,Highway,Bus,23.277057,45.033102,7.932627,7.852163,2472.346181
2,2022-08-01 00:58:00,0.564524,Rainy,Highway,Bicycle,33.671933,56.902619,10.259033,0.078435,9963.200425
3,2022-08-01 01:00:00,0.604874,Cloudy,Highway,Car,17.250465,77.853605,17.109540,9.755478,9890.142818
4,2022-08-01 01:18:00,0.284802,Clear,Highway,Bicycle,27.079891,32.905906,0.410464,1.794691,3806.339239
...,...,...,...,...,...,...,...,...,...,...
32898,2023-08-01 22:20:00,0.676100,Rainy,Highway,Bus,15.877835,86.444476,2.173862,1.035171,1935.623554
32899,2023-08-01 22:40:00,0.145728,Cloudy,City Street,Bicycle,29.187865,47.678940,1.948218,8.298239,4463.439080
32900,2023-08-01 23:00:00,0.414554,Rainy,Highway,Bus,34.560057,24.227049,5.226511,7.806714,646.823662
32901,2023-08-01 23:27:00,0.174059,Clear,Highway,Bus,24.384366,38.586769,10.789261,9.615835,9178.713445


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32903 entries, 0 to 32902
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   timestamp           32903 non-null  datetime64[ns]
 1   congestion_level    32903 non-null  float64       
 2   weather_conditions  32903 non-null  object        
 3   road_type           32903 non-null  object        
 4   vehicle_type        32903 non-null  object        
 5   temperature         32903 non-null  float64       
 6   humidity            32903 non-null  float64       
 7   wind_speed          32903 non-null  float64       
 8   precipitation       32903 non-null  float64       
 9   population_density  32903 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 2.5+ MB


### analyze data set

In [26]:
# convert/cast  Time col to index of datetime type
df['timestamp']=pd.to_datetime(df['timestamp'], infer_datetime_format=True)

df.set_index('timestamp')[['temperature','wind_speed','population_density','humidity']]
df.dtypes


C:\Users\nane\AppData\Local\Temp\ipykernel_16836\4235405620.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['timestamp']=pd.to_datetime(df['timestamp'], infer_datetime_format=True)


timestamp             datetime64[ns]
congestion_level             float64
weather_conditions            object
road_type                     object
vehicle_type                  object
temperature                  float64
humidity                     float64
wind_speed                   float64
precipitation                float64
population_density           float64
dtype: object

In [38]:
features = df[['congestion_level','temperature','population_density']]
target = df[['congestion_level']]
features

,congestion_level,temperature,population_density
0,0.843396,26.375612,3459.059905
1,0.858039,23.277057,2472.346181
2,0.564524,33.671933,9963.200425
3,0.604874,17.250465,9890.142818
4,0.284802,27.079891,3806.339239
...,...,...,...
32898,0.676100,15.877835,1935.623554
32899,0.145728,29.187865,4463.439080
32900,0.414554,34.560057,646.823662
32901,0.174059,24.384366,9178.713445


In [41]:
features.describe()

,congestion_level,temperature,population_density
count,32903.000000,32903.000000,32903.000000
mean,0.500828,24.981923,5248.454026
std,0.231102,5.749231,2738.335243
min,0.100046,15.000051,500.146734
25%,0.300458,19.985561,2884.094925
50%,0.500419,25.001625,5258.643157
75%,0.701496,29.930345,7600.252731
max,0.899959,34.999319,9999.993006


In [1]:
#ts_generator[0]